In [2]:
import pandas as pd
import re
from haversine import haversine
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

import numpy as np


In [3]:
df = pd.read_csv('dados/train.csv')
df1 = df.copy()

# Transformação de Dados

### Leitura do dados

In [4]:
df1.head(20)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2.0,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2.0,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0.0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0.0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1.0,Snack,scooter,1,No,Metropolitian,(min) 30
5,0x9bb4,HYDRES09DEL03,22,4.8,17.431668,78.408321,17.461668,78.438321,11-03-2022,21:20:00,21:30:00,conditions Cloudy,Jam,0.0,Buffet,motorcycle,1,No,Urban,(min) 26
6,0x95b4,RANCHIRES15DEL01,33,4.7,23.369746,85.339820,23.479746,85.449820,04-03-2022,19:15:00,19:30:00,conditions Fog,Jam,1.0,Meal,scooter,1,No,Metropolitian,(min) 40
7,0x9eb2,MYSRES15DEL02,35,4.6,12.352058,76.606650,12.482058,76.736650,14-03-2022,17:25:00,17:30:00,conditions Cloudy,Medium,2.0,Meal,motorcycle,1,No,Metropolitian,(min) 32
8,0x1102,HYDRES05DEL02,22,4.8,17.433809,78.386744,17.563809,78.516744,20-03-2022,20:55:00,21:05:00,conditions Stormy,Jam,0.0,Buffet,motorcycle,1,No,Metropolitian,(min) 34
9,0xcdcd,DEHRES17DEL01,36,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55:00,22:10:00,conditions Fog,Jam,2.0,Snack,motorcycle,3,No,Metropolitian,(min) 46


In [5]:
df1.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition              float64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

### Remover espaços vazios strip em toda a tabela

In [6]:
tipos_de_dados = df.dtypes
df1 = df1.reset_index(drop=True) # Quando remove-se linhas ele também mata a linha do index.
for column, tipo in tipos_de_dados.items():
    if tipo == 'object':
        df1[column] = df[column].str.strip()


### Remover linhas NaN de várias colunas

In [7]:
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN'
df1 = df1.loc[linhas_selecionadas, :].copy()
df1.dropna(subset=['Delivery_person_Age'], inplace=True)

linhas_selecionadas = df1['multiple_deliveries'] != 'NaN'
df1 = df1.loc[linhas_selecionadas, :].copy()
df1.dropna(subset=['multiple_deliveries'], inplace=True)

linhas_selecionadas = df1['Road_traffic_density'] != 'NaN'
df1 = df1.loc[linhas_selecionadas, :].copy()
df1.dropna(subset=['Road_traffic_density'], inplace=True)

linhas_selecionadas = df1['City'] != 'NaN'
df1 = df1.loc[linhas_selecionadas, :].copy()
df1.dropna(subset=['City'], inplace=True)

linhas_selecionadas = df1['Festival'] != 'NaN'
df1 = df1.loc[linhas_selecionadas, :].copy()
df1.dropna(subset=['Festival'], inplace=True)


### Removendo o texto inteiro da coluna de Time_taken

In [8]:
df1['Time_taken(min)'] = df1['Time_taken(min)'].str.replace(r'\(min\)', '', regex=True)

### Converter colunas

In [9]:
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

# Analise de Dados

### Quantidade de Entregas

In [10]:
df1.shape[0]
cols = ['ID', 'Order_Date']
df_aux = df1.loc[:, cols].groupby('Order_Date').count().reset_index()
media = np.mean(df_aux['ID'])
fig = px.bar(df_aux, x='Order_Date', y='ID', color='ID')
fig.add_hline(y=media, line_dash='dash', line_color='red', annotation_text=f'Média: {media:.2f}')

### Quantidade de Entregadores

In [11]:
delivery_unique = len(df1.loc[:, 'Delivery_person_ID'].unique())
print(f'A quantidade de entregadores é: {delivery_unique}')

A quantidade de entregadores é: 1244


### Pedidos por tipo de tráfego

In [12]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()
df_aux['entregas_perc'] = df_aux['ID']/df_aux['ID'].sum()
fig = px.pie(df_aux, values='entregas_perc', names='Road_traffic_density')

### Pedidos por cidade e tipo de tráfego

In [13]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
fig = px.bar(df_aux, x='Road_traffic_density', y='ID', color='City')
#fig = px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID')

### Locazação de cada cidade por tipo de tráfego

In [14]:
import folium

df_aux =df1.loc[:,['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City','Road_traffic_density']).median().reset_index()
map = folium.Map(location=[-23.54, -46.63],zoom_start=1)

for index, location_info in df_aux.iterrows():
    folium.Marker( [location_info['Delivery_location_latitude'],
                    location_info['Delivery_location_longitude']],
                    popup=location_info[['City', 'Road_traffic_density']]).add_to(map)

### Pedidos por semana

In [15]:
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')
df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

### Pedidos por entregador por semana

In [16]:
df_aux01 = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux02 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux01, df_aux02)
df_aux['order_by_delivery'] = df_aux['ID'] / df_aux['Delivery_person_ID']

### Idade do entregador mais velho e mais novo

In [17]:
velho = df1['Delivery_person_Age'].max()
novo = df1['Delivery_person_Age'].min()
print(f'O entregador mais velho tem {velho} anos, e o mais novo tem {novo} anos.')

O entregador mais velho tem 39 anos, e o mais novo tem 20 anos.


### ID dos entregadores mais velho e o mais novo

In [18]:
#df1.sort_values(['Delivery_person_Age'], ascending=False)
id_mais_velho = df1.loc[df1['Delivery_person_Age'].idxmax()]['ID']
id_mais_novo = df1.loc[df1['Delivery_person_Age'].idxmin()]['ID']
print(f'ID do entregador mais novo é {id_mais_novo} e do mais velho é {id_mais_velho}')

ID do entregador mais novo é 0xb3a0 e do mais velho é 0xddc5


### Quais as condições climáticas?

In [19]:
df1['Weatherconditions'].unique()

array(['conditions Sunny', 'conditions Stormy', 'conditions Sandstorms',
       'conditions Cloudy', 'conditions Fog', 'conditions Windy'],
      dtype=object)

### Quantas entregas foram realizados sob condições climáticas Sandstorm

In [20]:
filtro = df1['Weatherconditions'] == 'conditions Sandstorms'
len(df1.loc[filtro, :])

1141

In [21]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2.0,Snack,motorcycle,0,No,Urban,24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2.0,Snack,scooter,1,No,Metropolitian,33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0.0,Drinks,motorcycle,1,No,Urban,26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0.0,Buffet,motorcycle,1,No,Metropolitian,21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1.0,Snack,scooter,1,No,Metropolitian,30,12


### Qual a condição da entrega mais recente

In [22]:
df1.loc[df1['Order_Date'].idxmax()]['Weatherconditions']

'conditions Sunny'

### Quais tipos de densidade de trânsito?

In [23]:
df1['Road_traffic_density'].unique()

array(['High', 'Jam', 'Low', 'Medium'], dtype=object)

### Quantas entregas foram feitas em cada condição climatica

In [24]:
condicoes = list(df1['Weatherconditions'].unique())

for condicao in condicoes:
    qtd_entregas = df1.loc[df1['Weatherconditions'] == condicao, :].shape[0]
    print(f'Foram feitas {qtd_entregas} na condição {condicao}')

Foram feitas 1135 na condição conditions Sunny
Foram feitas 1184 na condição conditions Stormy
Foram feitas 1141 na condição conditions Sandstorms
Foram feitas 1182 na condição conditions Cloudy
Foram feitas 1216 na condição conditions Fog
Foram feitas 1186 na condição conditions Windy


### Quantos entregadores fizeram entregas em cada condição climática

In [25]:
condicoes = list(df1['Weatherconditions'].unique())

for condicao in condicoes:
    qnt_entregadores = len(df1.loc[df1['Weatherconditions'] == condicao, 'Delivery_person_ID'].unique())
    print(f'A condição climática {condicao} teve o total de {qnt_entregadores} entregadores')

A condição climática conditions Sunny teve o total de 713 entregadores
A condição climática conditions Stormy teve o total de 680 entregadores
A condição climática conditions Sandstorms teve o total de 689 entregadores
A condição climática conditions Cloudy teve o total de 729 entregadores
A condição climática conditions Fog teve o total de 722 entregadores
A condição climática conditions Windy teve o total de 707 entregadores


### Quantas entregas foram feitas em cada tipo de densidade de trânsito

In [26]:
condicoes = list(df1['Road_traffic_density'].unique())

for condicao in condicoes:
    qnt_entregas = df1.loc[df1['Road_traffic_density'] == condicao, :].shape[0]
    print(f'A condição de trânsito {condicao} teve o total de {qnt_entregas} entregas')

A condição de trânsito High teve o total de 717 entregas
A condição de trânsito Jam teve o total de 2279 entregas
A condição de trânsito Low teve o total de 2339 entregas
A condição de trânsito Medium teve o total de 1709 entregas


### Quantos entregadores únicos fizeram entregas em cada tipo de densidade de trânsito

In [27]:
condicoes = list(df1['Road_traffic_density'].unique())

for condicao in condicoes:
    qnt_entregadores = len(df1.loc[df1['Road_traffic_density'] == condicao, 'Delivery_person_ID'].unique())
    print(f'A condição de trânsito {condicao} teve o total de {qnt_entregadores} entregadores únicos')

A condição de trânsito High teve o total de 407 entregadores únicos
A condição de trânsito Jam teve o total de 959 entregadores únicos
A condição de trânsito Low teve o total de 961 entregadores únicos
A condição de trânsito Medium teve o total de 842 entregadores únicos


### Quantos tipos de cidade unicas e quais seus nomes

In [28]:
cidades = df1['City'].unique()
qnt_cidades = len(cidades)
print(f'Existem {qnt_cidades} cidades e são: {cidades}')

Existem 3 cidades e são: ['Urban' 'Metropolitian' 'Semi-Urban']


### Quantos tipos de veículos existem

In [29]:
qnt_veiculos = len(df1['Type_of_vehicle'].unique())
print(f'Existem {qnt_veiculos} tipo de veiculos')

Existem 3 tipo de veiculos


### Qual o tipo de veiculo que mais fez entregas com trânsito pesado

In [30]:
veiculos = list(df1['Type_of_vehicle'].unique())
total_entregas = 0
tipo_veiculos = ''

for veiculo in veiculos:
    filtro = (df1['Type_of_vehicle'] == veiculo) & (df1['Road_traffic_density'] == 'High')
    aux = df1.loc[filtro, :].shape[0]

    if aux > total_entregas:
        total_entregas = aux
        tipo_veiculo = veiculo

print(f'{tipo_veiculo} -> {total_entregas}')

motorcycle -> 427


### Qual o tipo de pedido mais feito durante condições climáticas de Sandstorms

In [31]:
pedidos = list(df1['Type_of_order'].unique())
total_entregas = 0
tipo_clima = ''

for pedido in pedidos:
    filtro = (df1['Type_of_order'] == pedido) & (df1['Weatherconditions'] == 'conditions Sandstorms')
    aux = df1.loc[filtro, :].shape[0]

    if aux > total_entregas:
        total_entregas = aux
        tipo_clima = pedido

print(f'{tipo_clima} -> {total_entregas}')


Meal -> 312


### Qual a cidade com o maior número de pedidos de bebidas (drinks) feito em scooter

In [32]:
cidades = list(df1['City'].unique())
total_entregas = 0
tipo_cidade = ''

for cidade in cidades:
    filtro = (df1['City'] == cidade) & (df1['Type_of_order'] == 'Drinks') & (df1['Type_of_vehicle'] == 'scooter')
    aux = df1.loc[filtro, :].shape[0]

    if aux > total_entregas:
        total_entregas = aux
        tipo_cidade = cidade

print(f'{tipo_cidade} -> {total_entregas}')

Metropolitian -> 453


### Nome da cidade com os entradores mais velhos e o nome da cidade os mais novos

# Distância dos Restaurantes

In [33]:
cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']
df1['distance'] = df1.loc[:, cols].apply( lambda row: haversine((row['Restaurant_latitude'], row['Restaurant_longitude']), (row['Delivery_location_latitude'], row['Delivery_location_longitude'])), axis=1)


# Retirando o (mim) da coluna de tempo e transformando em numérico

In [34]:
#df1['Time_taken(min)'] = df1['Time_taken(min)'].replace(r'\D', '', regex=True)
#df1['Time_taken(min)'] = df1.to_numeric(df1['Time_taken(min)'])

# Entregadores mais novos e velhos

In [35]:
df_aux_max = df1.loc[:, 'Delivery_person_Age'].max()
df_aux_min = df1.loc[:, 'Delivery_person_Age'].min()

print(f'O entregador com mais idade tem: {df_aux_max} anos')
print(f'O entregador com menos ideade tem: {df_aux_min} anos')

O entregador com mais idade tem: 39 anos
O entregador com menos ideade tem: 20 anos


###

# Condição de veículos mais novo e o pior

In [36]:
df_aux_cond_vei_max = df1.loc[:, 'Vehicle_condition'].max()
df_aux_cond_vei_min = df1.loc[:, 'Vehicle_condition'].min()

print(f'O veiculo mais novo tem nota {df_aux_cond_vei_max}')
print(f'O veiculo mais velho tem nota {df_aux_cond_vei_min}')


O veiculo mais novo tem nota 2.0
O veiculo mais velho tem nota 0.0


# Avaliação média por Entregador

In [37]:
df_aux_med_ent = df1.loc[:, ['Delivery_person_Ratings', 'Delivery_person_ID']].groupby('Delivery_person_ID').mean().reset_index()
df_aux_med_ent

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.700000
1,AGRRES010DEL02,4.700000
2,AGRRES010DEL03,4.500000
3,AGRRES01DEL02,4.550000
4,AGRRES01DEL03,4.733333
...,...,...
1239,VADRES19DEL02,4.576923
1240,VADRES19DEL03,4.644444
1241,VADRES20DEL01,4.650000
1242,VADRES20DEL02,4.430000


# Avaliação média e Desvio Padrão por tipo de tráfego

In [38]:
df_aux_med_desv_tra = df1.loc[:, ['Delivery_person_Ratings', 'Road_traffic_density']].groupby('Road_traffic_density').agg({'Delivery_person_Ratings': ['mean', 'std']})


# Mudar nomes das colunas
df_aux_med_desv_tra.columns = ['Média', 'Desvio Pad']
df_aux_med_desv_tra = df_aux_med_desv_tra.reset_index(drop=True)

df_aux_med_desv_tra


,Média,Desvio Pad
0,4.662762,0.269895
1,4.593805,0.322192
2,4.633390,0.345727
3,4.654396,0.280000


# Avaliação média e Desvio Padrão por tipo de Clima

In [39]:
df_aux_med_desv_cli = df1.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']].groupby('Weatherconditions').agg({'Delivery_person_Ratings': ['mean', 'std']})

# Mudar nomes das colunas
df_aux_med_desv_cli.columns = ['Média', 'Desvio Pad']
df_aux_med_desv_cli = df_aux_med_desv_cli.reset_index(drop=True)

df_aux_med_desv_cli


,Média,Desvio Pad
0,4.638762,0.281638
1,4.653947,0.266977
2,4.600615,0.320025
3,4.609206,0.311229
4,4.646337,0.406090
5,4.622213,0.298440


# Velocidade de Entregadores

## Entregadores mais rápidos

In [40]:
df_ent_mais_rap = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['City', 'Time_taken(min)'], ascending=True).reset_index()

df_aux01 = df_ent_mais_rap.loc[df_ent_mais_rap['City'] == 'Metropolitian', :].head(10)
df_aux02 = df_ent_mais_rap.loc[df_ent_mais_rap['City'] == 'Urban', :].head(10)
df_aux03 = df_ent_mais_rap.loc[df_ent_mais_rap['City'] == 'Semi-Urban', :].head(10)

df_ent_mais_rapido = pd.concat( [df_aux01, df_aux02, df_aux03]).reset_index(drop=True)


## Entregadores mais lentos

In [41]:
df_ent_mais_len = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['City', 'Time_taken(min)'], ascending=False).reset_index()

df_aux01 = df_ent_mais_len.loc[df_ent_mais_len['City'] == 'Metropolitian', :].head(10)
df_aux02 = df_ent_mais_len.loc[df_ent_mais_len['City'] == 'Urban', :].head(10)
df_aux03 = df_ent_mais_len.loc[df_ent_mais_len['City'] == 'Semi-Urban', :].head(10)

df_ent_mais_len = pd.concat( [df_aux01, df_aux02, df_aux03]).reset_index(drop=True)
df_ent_mais_len


,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,LUDHRES19DEL03,51.0
1,Metropolitian,BHPRES05DEL01,49.0
2,Metropolitian,AGRRES02DEL01,48.0
3,Metropolitian,AGRRES13DEL01,48.0
4,Metropolitian,ALHRES18DEL02,48.0
5,Metropolitian,KOCRES12DEL01,48.0
6,Metropolitian,AGRRES02DEL02,47.0
7,Metropolitian,AGRRES20DEL02,46.0
8,Metropolitian,GOARES13DEL01,46.0
9,Metropolitian,PUNERES19DEL03,46.0


# Quantidade de Entregadores

In [42]:
qnt_entregadores = len(df1.loc[:, 'Delivery_person_ID'].unique())
print(f'A quantidade de entregadores é {qnt_entregadores}')

A quantidade de entregadores é 1244


# Distancia Média das Entregas

In [43]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df1['distance'] = df1.apply(lambda x: haversine((x['Restaurant_latitude'], x['Restaurant_longitude']),
                                               (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1)

dist_media = np.round(df1['distance'].mean(), 2)

dist_media

30.13

# Tempo médio de entrega durante Eventos

In [44]:
cols = ['Time_taken(min)', 'Festival']
temp_med_fest = df1.loc[:, cols].groupby('Festival').agg({'Time_taken(min)': ['mean', 'std']})

temp_med_fest.columns = ['Tempo Medio', 'Desv Padrao']
temp_med_fest = temp_med_fest.reset_index()
temp_med_fest = np.round(temp_med_fest.loc[temp_med_fest['Festival'] == 'Yes', 'Tempo Medio'], 2)


# Função para formatar minutos e segundos
def formatar_tempo(media_em_minutos):
    minutos = int(media_em_minutos)
    segundos = round((media_em_minutos - minutos) * 60)
    return timedelta(minutes=minutos, seconds=segundos)

# Certifique-se de que a Série está convertida para o tipo correto
temp_med_fest = temp_med_fest.astype(float)

# Usando a função map para aplicar a formatação
temp_med_fest_formatada = temp_med_fest.map(formatar_tempo)

# Converter a Série para uma lista de strings formatadas
temp_med_fest_lista = temp_med_fest_formatada.apply(lambda x: f"{(x.seconds // 60) % 60:02d}:{x.seconds % 60:02d}")

# Selecione o primeiro valor da lista para usar no método metric
valor_metric = temp_med_fest_lista.iloc[0]

valor_metric



'44:59'

# Desvio Padrão do tempo de entrega durante Eventos

In [45]:
cols = ['Time_taken(min)', 'Festival']
temp_med_fest = df1.loc[:, cols].groupby('Festival').agg({'Time_taken(min)': ['mean', 'std']})

temp_med_fest.columns = ['Tempo Medio', 'Desv Padrao']
temp_med_fest = temp_med_fest.reset_index()
temp_med_fest = np.round(temp_med_fest.loc[temp_med_fest['Festival'] == 'Yes', 'Desv Padrao'], 2)


# Função para formatar minutos e segundos
def formatar_tempo(media_em_minutos):
    minutos = int(media_em_minutos)
    segundos = round((media_em_minutos - minutos) * 60)
    return timedelta(minutes=minutos, seconds=segundos)

# Certifique-se de que a Série está convertida para o tipo correto
temp_med_fest = temp_med_fest.astype(float)

# Usando a função map para aplicar a formatação
temp_med_fest_formatada = temp_med_fest.map(formatar_tempo)

# Converter a Série para uma lista de strings formatadas
temp_med_fest_lista = temp_med_fest_formatada.apply(lambda x: f"{(x.seconds // 60) % 60:02d}:{x.seconds % 60:02d}")

# Selecione o primeiro valor da lista para usar no método metric
valor_metric = temp_med_fest_lista.iloc[0]

temp_med_fest



1    3.97
Name: Desv Padrao, dtype: float64

# Tempo Médio de Entrega por Cidade

In [46]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df1['distance'] = df1.apply(lambda x: haversine((x['Restaurant_latitude'], x['Restaurant_longitude']),
                                               (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1)

tempo_medio_cidades = df1.loc[:, ['City', 'distance']].groupby('City').mean().reset_index()
tempo_medio_cidades


,City,distance
0,Metropolitian,30.808076
1,Semi-Urban,12.544586
2,Urban,28.107670


#

# Tempo Médio com Desv Padrão

In [47]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df_aux = df1.loc[:, ['City', 'Time_taken(min)']].groupby('City').agg({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['Tempo Medio', 'Desv Padrao']
df_aux = df_aux.reset_index()

df_aux

,City,Tempo Medio,Desv Padrao
0,Metropolitian,27.671028,9.151402
1,Semi-Urban,49.851852,2.741472
2,Urban,22.994343,8.951826


# Distribuição do Tempo

In [63]:
df_aux = df1.loc[:, ['City', 'Time_taken(min)']].groupby('City').agg({'Time_taken(min)':['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']
df_aux = df_aux.reset_index()

fig = go.Figure()

# Utilize hovertemplate para personalizar o texto que aparece quando você passa o mouse sobre as barras
hover_template = '%{x}: %{y:.6f} ± %{customdata:.2f}'
fig.add_trace(go.Bar(
    name='',
    x=df_aux['City'],
    y=df_aux['avg_time'],
    error_y=dict(type='data', array=df_aux['std_time']),
    customdata=df_aux['std_time'],
    hovertemplate=hover_template
))

fig.update_layout(barmode='group')
fig.show()


# Tempo Médio e Desvio Padrão de entrega por cidade

In [79]:
cols =['City', 'Time_taken(min)', 'Road_traffic_density']
df_aux = df1.loc[:, cols].groupby(['City', 'Road_traffic_density']).agg({'Time_taken(min)' : ['mean', 'std']})

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

fig = px.sunburst(df_aux, 
                  path=['City', 'Road_traffic_density'], 
                  values='avg_time',
                  color='std_time',
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df_aux['std_time']))

# Personalizando o layout da dica de ferramenta
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>Tempo Médio: %{value:.2f} minutos<br>Std Time: %{customdata[0]:.2f} minutos'
)

# Ajustando o título da barra de cores
fig.update_layout(coloraxis_colorbar_title='Desvio Padrão de Tempo (minutos)')

fig.show()

# Distribuição da Distância por Tipo de Alimento

In [85]:
cols = ['City', 'Time_taken(min)', 'Type_of_order']
df_aux = df1.loc[:, cols].groupby(['City', 'Type_of_order']).agg({'Time_taken(min)': ['mean', 'std']})

df_aux.columns = ['Tempo Medio (min)', 'Desvio Padrão (min)']

df_aux = df_aux.reset_index()

df_aux = df_aux.rename(columns={'City': 'Cidade', 'Type_of_order': 'Tipo de Pedido'})

df_aux

,Cidade,Tipo de Pedido,Tempo Medio (min),Desvio Padrão (min)
0,Metropolitian,Buffet,27.676606,9.417227
1,Metropolitian,Drinks,27.368960,8.853055
2,Metropolitian,Meal,27.542530,9.066485
3,Metropolitian,Snack,28.103474,9.267615
4,Semi-Urban,Buffet,50.250000,2.915476
5,Semi-Urban,Drinks,52.250000,2.217356
6,Semi-Urban,Meal,49.666667,3.386247
7,Semi-Urban,Snack,48.555556,1.740051
8,Urban,Buffet,22.835821,8.984770
9,Urban,Drinks,23.524876,9.244295
